## Ejemplos usando CuPy

In [ ]:
# Vemos si tenemos instalada la librería CuPy
import numpy as np
import cupy as cp

## Ejemplo 1: Declaración de variables en CuPy

In [ ]:
#Declaramos un arreglo en la CPU
x_cpu=np.array([1,2,3])

#Declaramos un arreglo en la GPU
x_gpu=cp.array([1,2,3])

#Calculamos la norma L2 en la CPU
norm_L2_x_cpu=np.linalg.norm(x_cpu)

#Calculamos la norma L2 en la GPU
norm_L2_x_GPU=cp.linalg.norm(x_gpu)

#Mostrar resultados
print(norm_L2_x_cpu)
print(norm_L2_x_GPU)

with cp.cuda.Device(0):
    y_gpu = cp.array([1, 2, 3, 4, 5])
print(y_gpu.device)

3.7416573867739413
3.7416573867739413
<CUDA Device 0>


## Ejemplo 2: Transferencia de datos

In [ ]:
x_cpu = np.array([1, 2, 3])

#Copiar datos desde la CPU a la GPU
x_gpu = cp.asarray(x_cpu)

#Copiar datos desde la GPU a la CPU
y_cpu = cp.asnumpy(x_gpu)
#y_cpu = x_gpu.get() #También se puede
                     #usar el atributo get()
#Mostrar resultados
print(x_cpu)
print(x_gpu)
print(y_cpu)

#cp.asarray y cp.asnumpy aceptan cualquier entrada (CPU/GPU)
#cp.asarray devuelve un arreglo CuPy en el device
#cp.asnumpy devuelve un arreglo Numpy en el host
z_cpu=x_cpu+y_cpu
#z_aux=x_gpu+y_cpu #No es posible
#Podemos hacer lo siguiente
z2_cpu=cp.asnumpy(x_gpu)+y_cpu
#o esto
z3_gpu=x_gpu+cp.asarray(y_cpu)

print(z_cpu)
print(z2_cpu)
print(z3_gpu)

[1 2 3]
[1 2 3]
[1 2 3]
[2 4 6]
[2 4 6]
[2 4 6]


## Ejemplo 3: CUDA-Kernel Elemental

In [ ]:
#Creando una función CUDA-Kernel para evaluar las diferencias al cuadrado
squared_diff = cp.ElementwiseKernel('float32 x, float32 y',
                                    'float32 z',
                                    'z = (x - y) * (x - y)',
                                    'squared_diff')

x = cp.arange(10, dtype=np.float32).reshape(2, 5)
y = cp.arange(5, dtype=np.float32)

#z=squared_diff(x, y)
#También se puede hacer esto
z = cp.empty((2, 5), dtype=np.float32)
squared_diff(x, y,z)

print(x)
print(y)
print(z)


[[0. 1. 2. 3. 4.]
 [5. 6. 7. 8. 9.]]
[0. 1. 2. 3. 4.]
[[ 0.  0.  0.  0.  0.]
 [25. 25. 25. 25. 25.]]


## Ejemplo 4: CUDA-Kernel Reducción

In [ ]:
l2norm_kernel = cp.ReductionKernel('T x', # Entrada
                                   'T y', # Salida
                                   'x * x', # 1- Mapeo
                                   'a + b', # 2- Reducción
                                   'y = sqrt(a)', # 3- Mapeo posterior
                                   '0', # 4- Valor inicial
                                   'l2norm' # Nombre del Kernel
                                   )

x = cp.arange(10, dtype=np.float32).reshape(2, 5)
y=l2norm_kernel(x, axis=1)

print(x)
print(y)

[[0. 1. 2. 3. 4.]
 [5. 6. 7. 8. 9.]]
[ 5.477226  15.9687195]


## Ejemplo 5: CUDA-Kernel Crudo

In [ ]:
VectorAdd_kernel_cp = cp.RawKernel(r'''
    extern "C" __global__
    void VectorAdd_kernel(const float* x1_d, const float* x2_d, float* y_d) {
        int tid = blockDim.x * blockIdx.x + threadIdx.x;
        y_d[tid] = x1_d[tid] + x2_d[tid];
    }
    ''', 'VectorAdd_kernel')
x1_d = cp.arange(25, dtype=cp.float32).reshape(5, 5)
x2_d = cp.arange(25, dtype=cp.float32).reshape(5, 5)
y_d = cp.zeros((5, 5), dtype=cp.float32)
VectorAdd_kernel_cp((5,), (5,), (x1_d, x2_d, y_d)) # grid, block and arguments
                                    # Notar que tanto el grid como el block son 1D
print(y_d)

[[ 0.  2.  4.  6.  8.]
 [10. 12. 14. 16. 18.]
 [20. 22. 24. 26. 28.]
 [30. 32. 34. 36. 38.]
 [40. 42. 44. 46. 48.]]


## Ejemplo 6: CUDA-Kernel Modulo

In [ ]:
loaded_from_source = r'''
extern "C"{
    __global__ void test_sum(const float* x1_d, const float* x2_d, float* y_d,unsigned int N){
        unsigned int tid = blockDim.x * blockIdx.x + threadIdx.x;
        if (tid < N){
            y_d[tid] = x1_d[tid] + x2_d[tid];
        }
    }
    __global__ void test_mult(const float* x1_d, const float* x2_d, float* y_d,unsigned int N){
        unsigned int tid = blockDim.x * blockIdx.x + threadIdx.x;
        if (tid < N){
            y_d[tid] = x1_d[tid] * x2_d[tid];
        }
    }
}'''
module = cp.RawModule(code=loaded_from_source)
ker_sum = module.get_function('test_sum')
ker_mult = module.get_function('test_mult')
N = 10
x1_d = cp.arange(N**2, dtype=cp.float32).reshape(N, N)
x2_d = cp.ones((N, N), dtype=cp.float32)*2.5
y_d  = cp.zeros((N, N), dtype=cp.float32)
z_d  = cp.zeros((N, N), dtype=cp.float32)
ker_sum((N,), (N,), (x1_d, x2_d, y_d, N**2))
ker_mult((N,), (N,), (x1_d, x2_d, z_d, N**2))
print(y_d)
print(z_d)

[[  2.5   3.5   4.5   5.5   6.5   7.5   8.5   9.5  10.5  11.5]
 [ 12.5  13.5  14.5  15.5  16.5  17.5  18.5  19.5  20.5  21.5]
 [ 22.5  23.5  24.5  25.5  26.5  27.5  28.5  29.5  30.5  31.5]
 [ 32.5  33.5  34.5  35.5  36.5  37.5  38.5  39.5  40.5  41.5]
 [ 42.5  43.5  44.5  45.5  46.5  47.5  48.5  49.5  50.5  51.5]
 [ 52.5  53.5  54.5  55.5  56.5  57.5  58.5  59.5  60.5  61.5]
 [ 62.5  63.5  64.5  65.5  66.5  67.5  68.5  69.5  70.5  71.5]
 [ 72.5  73.5  74.5  75.5  76.5  77.5  78.5  79.5  80.5  81.5]
 [ 82.5  83.5  84.5  85.5  86.5  87.5  88.5  89.5  90.5  91.5]
 [ 92.5  93.5  94.5  95.5  96.5  97.5  98.5  99.5 100.5 101.5]]
[[  0.    2.5   5.    7.5  10.   12.5  15.   17.5  20.   22.5]
 [ 25.   27.5  30.   32.5  35.   37.5  40.   42.5  45.   47.5]
 [ 50.   52.5  55.   57.5  60.   62.5  65.   67.5  70.   72.5]
 [ 75.   77.5  80.   82.5  85.   87.5  90.   92.5  95.   97.5]
 [100.  102.5 105.  107.5 110.  112.5 115.  117.5 120.  122.5]
 [125.  127.5 130.  132.5 135.  137.5 140.  142.5 145.